In [3]:
pip install asyncpraw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 4.3 MB/s eta 0:00:00


In [5]:
pip install nest_asyncio

In [ ]:
import asyncpraw
import pandas as pd
import csv
from datetime import datetime, timezone
import asyncio
import nest_asyncio

nest_asyncio.apply()

def get_reddit_credentials():
    client_id = input("Client ID: ")
    client_secret = input("Client Secret: ")
    user_agent = input("User Agent: ")
    return client_id, client_secret, user_agent

def create_reddit_instance(client_id, client_secret, user_agent):
    return asyncpraw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        user_agent=user_agent
    )

def save_to_csv(data, filename):
    if not data:
        print("The data list is empty. Cannot save to CSV.")
        return

    keys = data[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)
    print(f"Posts saved to {filename}")

def append_submission_data(data, submission):
    data.append({
        'author': str(submission.author),
        'title': submission.title,
        'number_of_comments': submission.num_comments,
        'date': datetime.fromtimestamp(submission.created_utc, tz=timezone.utc),
        'content': submission.selftext if submission.is_self else None,
        'url': submission.url
    })

async def fetch_posts_with_target_word(reddit, target_word, limit):
    data = []
    subreddit = await reddit.subreddit('all')
    async for submission in subreddit.search(query=target_word, sort='new', limit=limit):
        append_submission_data(data, submission)
    return data

async def main():
    client_id, client_secret, user_agent = get_reddit_credentials()
    reddit = create_reddit_instance(client_id, client_secret, user_agent)
    target_word = input("Target word in title: ")
    limit = 2000
    data = await fetch_posts_with_target_word(reddit, target_word, limit)

    df = pd.DataFrame(data)
    filename = input("Enter filename (with .csv extension): ")
    save_to_csv(data, filename)
    print(df.head())

asyncio.run(main())